In [ ]:
from operator import itemgetter

from dotenv import load_dotenv
from langchain import hub
from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain_openai import ChatOpenAI

load_dotenv()

<img src="../images/multi-agent-supervisor.png" width="850" height="500">

In [ ]:
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, HumanMessagePromptTemplate, \
    ChatMessagePromptTemplate, PromptTemplate
from langchain_core.tools import tool

# Define the tools
@tool
def send_to_researcher(research_query: str) -> str:
    """Use this tool when you want to send task to the researcher with a research query"""
    return "Researcher"

@tool
def send_to_blogger(blogging_task: str) -> str:
    """Use this tool when you want to send task to the blogger with a blogging task"""
    return "Blogger"

@tool
def finish() -> str:
    """When you have the blog post ready, use this tool to finish the task."""
    return "Finish"

supervisor_tools = [send_to_researcher, send_to_blogger, finish]
llm_supervisor = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0).bind_functions(supervisor_tools)

prompt = PromptTemplate.from_template("""
# Instructions
You are a supervisor tasked with managing Researcher and Blogger an orchestrate the tasks between them and output the best blog post.
You should evaluate the research findings and the blog post in a critique eye and decide if the quality is enough.

You should send task to the researcher when the research is not complete which mean the findings of the researcher still missing information for writing the blog post.
The task to the reasearcher should be to search the web and to research about the topic it should be short and concise (maximum 15 words) and should guide the researcher to the right direction to start the research or to extend it.
The task for the blogger should be to write the blog post, it should be short and concise (maximum 15 words) and should guide the blogger what to write about or how to improve the existing blog post draft.
In any case you think the research materials are missing you should send task to the researcher, however if the research materials are complete but the blog post is still not ready from your perspective you should send task to the blogger.
In case the blog post is ready you should finish.
                                      
# Guidelines
**Extensive Research**: Make sure the researcher has done extensive research on the topic with meaningful findings.
**Quality Blog Post**: Make sure the blog post is well written and informative.

Topic: {topic}                        
Research Findings: {researcher_findings}
Blogger Draft: {blogger_draft}

Decide who should act next and on what task, Or should we FINISH?
""")
llm = ChatOpenAI(model="gpt-4-turbo-preview", temperature=0, verbose=True)
llm_gpt35 = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0)
supervisor_llm_with_tools = llm.bind(tools=[convert_to_openai_tool(tool) for tool in supervisor_tools])

supervisor = {"researcher_findings":  itemgetter("researcher_findings"), 
              "blogger_draft": itemgetter("blogger_draft"),
              "topic": itemgetter("topic")} | prompt | supervisor_llm_with_tools | OpenAIToolsAgentOutputParser()


In [ ]:
action = supervisor.invoke({"topic":"The impact of climate change on the economy", 
                            "researcher_findings":"Climate change is a major threat to the economy",
                            "blogger_draft":""})[0]
print(f"Name of function: {action.tool}")
print(f"Param: {action.tool_input}")

<img src="../images/multi-agent-researcher.png" width="850" height="500">

In [ ]:
prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.messages

In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are a researcher you can use the web to research about a task taking into account all the information you have on the subject."),
        ("human","Previous known information about the subject: {previous_information}"),
        ("human","Task: {task}"),
        ("human","Given the task and the previous known info if you think that you need more information to complete the research and present findings for the research task search the web using DuckDuckGo to find more information about the task. summarize your finding."),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ]
)

researcher_tools = [DuckDuckGoSearchRun()]
researcher_agent = ({
                 "previous_information": itemgetter("researcher_findings") | RunnableLambda(lambda x: "\n\n".join(x)),
                 "task": itemgetter("task"),
                "agent_scratchpad": itemgetter("intermediate_steps") | RunnableLambda(format_to_openai_tool_messages)
              } 
              | prompt
              | llm.bind(tools=[convert_to_openai_tool(tool) for tool in researcher_tools])
              | OpenAIToolsAgentOutputParser())
researcher = AgentExecutor(agent=researcher_agent, tools=researcher_tools, verbose=True)

In [ ]:
[convert_to_openai_tool(tool) for tool in researcher_tools]

In [ ]:
researcher_output = researcher.invoke({"researcher_findings":["Climate change is a major threat to the economy"], "task":"Research the impact of climate change on the economy"})
print(researcher_output)

<img src="../images/multi-agent-blogger.png" width="850" height="500">

In [ ]:
prompt = PromptTemplate.from_template("""
# Instructions
You are a professional blogger tasked with writing a blog post about a subject. you are given a research materials from a professional researcher that already did the research for you. Use ONLY the research materials to write the blog post and DON'T use your previous knowledge about the subject.
Research Findings:\n {researcher_findings}
Your previous Draft:\n {blogger_draft}
Current Writing Task\n: {current_task}
Your Blog Post:\n
""")
writer = (
    {
     "researcher_findings": itemgetter("researcher_findings"),
     "blogger_draft": itemgetter("blogger_draft"),
     "current_task": itemgetter("task")
    }
    | prompt
    | llm_gpt35
)

In [ ]:
writer.invoke({"researcher_findings":researcher_output['output'], "blogger_draft":"", "task":"Write a blog post about the impact of climate change on the economy"})

<img src="../images/multi-agent-state.png" width="850" height="500">

In [ ]:
import operator
from typing import Annotated, List, TypedDict
from langgraph.graph import StateGraph, END
from langchain.schema import AgentFinish


class AgentState(TypedDict):
    topic: str
    turn: str
    researcher_task: str
    blogger_task: str
    blogger_draft: str
    researcher_findings: Annotated[List[str], operator.add]

def supervisor_node(state):
    action = supervisor.invoke(state)
    if type(action) == AgentFinish:
        return {"turn":"finish"}
    action = action[0]
    if "send_to_researcher" == action.tool:
        return {"turn":"researcher", "researcher_task":action.tool_input , "researcher_findings": state["researcher_findings"] or [] }
    elif "send_to_blogger" == action.tool:
        return {"turn":"blogger", "blogger_task":action.tool_input, "researcher_findings": state["researcher_findings"] or [], "blogger_draft": state["blogger_draft"] or ""}
    else:
        return {"turn":"finish"}
    
def researcher_node(state):
    researcher_output = researcher.invoke({"researcher_findings":state["researcher_findings"], "task":state["researcher_task"]})
    return {"researcher_findings": [researcher_output['output']]}

def blogger_node(state):
    blogger_output = writer.invoke({"researcher_findings":state["researcher_findings"], "blogger_draft":state["blogger_draft"], "task":state["blogger_task"]})
    return {"blogger_draft": blogger_output.content}

workflow = StateGraph(AgentState)
workflow.add_node("researcher", researcher_node)
workflow.add_node("blogger", blogger_node)
workflow.add_node("supervisor", supervisor_node)
workflow.add_edge("researcher", "supervisor")
workflow.add_edge("blogger", "supervisor")

def routing_logic(state):
    if state["researcher_findings"] and len(state["researcher_findings"]) > 3 and state["turn"]=="researcher":
        return "blogger"
    else:
        return state["turn"]

conditional_map = {k:k for k in ["researcher", "blogger"]}
conditional_map["finish"] = END
workflow.add_conditional_edges("supervisor", routing_logic, conditional_map)
workflow.set_entry_point("supervisor")

graph = workflow.compile()
graph.invoke({"topic":"The impact of climate change on the economy"})

In [ ]:

for s in graph.stream(
    {"topic":"The impact of climate change on the economy"}
):
    if "__end__" not in s:
        print(s)
        print("----")